In [1]:
#train_E6oV3lV
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


Using TensorFlow backend.


In [2]:
data = pd.read_csv('train_E6oV3lV.csv') # read CSV file

In [7]:
#pre processing
data = data[['label','tweet']]
data['tweet'] = data['tweet'].apply(lambda x: x.lower())
data['tweet'] = data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
print(data[ data['label'] == 0].size)
print(data[ data['label'] == 1].size)


59440
4484


In [12]:
#padding 
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['tweet'].values)
X = tokenizer.texts_to_sequences(data['tweet'].values)
X = pad_sequences(X)

In [54]:
# sentiment analysis using tweetter dataset
#Deep Learning LSTM 
embed_dim = 128
lstm_out = 196
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 32, 128)           256000    
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 32, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [55]:
Y = data['label']
Y = to_categorical(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(21414, 32) (21414, 2)
(10548, 32) (10548, 2)


In [56]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
 - 71s - loss: 0.1855 - acc: 0.9407
Epoch 2/7
 - 68s - loss: 0.1234 - acc: 0.9568
Epoch 3/7
 - 67s - loss: 0.1058 - acc: 0.9613
Epoch 4/7
 - 69s - loss: 0.0923 - acc: 0.9655
Epoch 5/7
 - 66s - loss: 0.0822 - acc: 0.9694
Epoch 6/7
 - 66s - loss: 0.0718 - acc: 0.9729
Epoch 7/7
 - 67s - loss: 0.0629 - acc: 0.9774


In [57]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.18
acc: 0.95


In [64]:
from sklearn.metrics import classification_report
y_pred=model.predict(X_test, batch_size=32,verbose = 2)
report = classification_report(Y_test, y_pred.round())
print(report)


              precision    recall  f1-score   support

           0       0.97      0.98      0.97      9806
           1       0.63      0.56      0.59       742

   micro avg       0.95      0.95      0.95     10548
   macro avg       0.80      0.77      0.78     10548
weighted avg       0.94      0.95      0.94     10548
 samples avg       0.95      0.95      0.95     10548



In [39]:
# prediction 
#test_tweets_anuFYb8
test = pd.read_csv('test_tweets_anuFYb8.csv')

In [47]:

#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(test['tweet'])
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=32, dtype='int32', value=0)
sentiment = model.predict_classes(twt,batch_size=32,verbose = 2)
#sentiment = model.predict(twt,batch_size=32,verbose = 2)


In [51]:
test['label']=sentiment
test.to_csv('test_predictions.csv ')